In [ ]:
from typing import Optional, Tuple, Dict, Iterator
from functools import partial
import pathlib
import shutil
import random
import logging
import copy
logging.disable(logging.WARNING)
import os
from datetime import datetime
from itertools import product

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import onnx
from onnxruntime import InferenceSession
from onnxruntime.quantization import quantize_static, QuantFormat, QuantType, CalibrationDataReader
from onnxruntime.quantization.shape_inference import quant_pre_process

<p>Set constants of the experiment</p>

In [ ]:
DATA_PATH = pathlib.Path("data")
OUTPUT_PATH = pathlib.Path("output_ipynb")
DENSE_UNITS = 64
EPOCHS = 1000
BATCH_SIZE = 32
PATIENCE = 100
MODEL_NAME = f'dense_{DENSE_UNITS}x{DENSE_UNITS // 2}.onnx'

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

<p>Define features extraction and data preprocessing utilities</p>

In [ ]:
def zcr(win: np.ndarray):
    """Zero-crossing rate.

    Args:
        win (np.ndarray): array of samples.

    Returns:
        ZCR value.
    """
    zcr = 0
    curr_sign = np.sign(win[0])
    for sample in win:
        if np.sign(sample) * curr_sign < 0:
            curr_sign = np.sign(sample)
            zcr += 1
    return zcr

def mcr(win: np.ndarray, axis: int = 0):
    """Mean-crossing rate along the specified axis.

    Args:
        win (np.ndarray): array of samples.
        axis (int): axis index.

    Returns:
        np.ndarray: MCR values along x, y, and z axes.
    """
    means = np.mean(win, axis=axis)
    feat_list = []
    if axis == 0:
        win = win.transpose()
    for ax, m in enumerate(means):
        mcr = 0
        detrended_win = win[ax] - m
        feat_list.append(zcr(detrended_win))

    return np.array(feat_list)
         
        
def compute_features(win: np.ndarray):
    """Compute the set of features of interest on the given window of signal.
        - mean: contains information about the device orientation.
        - variance: contains information about the intensity of the activity.
        - mean-crossing rate: contains information about the fundamental frequency of the activity.

    Args:
        win (np.ndarray): array of samples.

    Returns:
        Tuple[np.ndarray, List[str]]: (3 x axes)-dimensional data-point (3 features x number of input axes), features name.
    """
    
    data_point = np.array([])
    features = [np.mean, np.var, mcr]
    for feat in features:
        data_point = np.concatenate([data_point, feat(win, axis=0)])

    return data_point, ['mean', 'var', 'mcr']

def load_dataset(
    data_path: str,
    win_len: int,
    win_stride: int = 1,
) -> Tuple[np.ndarray, np.ndarray, Dict[int, str]]:
    """Read CSV logs, discard columns, and segment data into windows.

    Args:
        data_path (str): Dataset path.
        win_len (int): Window length (#samples).
        win_stride (int): Window stride (#samples). Defaults to 1.

    Returns:
        Tuple[np.ndarray, np.ndarray, Dict[int, str], Dict[int, str]]:
            Features, labels, labels_dict, features_dict (index, name).
    """
    X = []
    y = []
    labels_dict: Dict[int, str] = {}
    for log_path in pathlib.Path(data_path).rglob("*.csv"):
        data_frame = pd.read_csv(log_path, sep=",", header=2, dtype=np.float32)
        label, activity = log_path.parent.name.split("_", maxsplit=1)
        label = int(label)
        labels_dict[label] = activity
        win_list = []
        idx = 0
        while idx < len(data_frame) - win_len:
            data_point, features_name = compute_features(data_frame.values[idx : idx + win_len])
            win_list.append(data_point)
            idx += win_stride
        X += win_list
        y += [label] * len(win_list)

    features_dict = {idx : '_'.join(t) for idx, t in enumerate(product(features_name, ['x', 'y', 'z']))}
    return (np.array(X), np.array(y), labels_dict, features_dict)


<p>Define training utilities</p>

In [ ]:
class FeaturesDataset(Dataset):
    """
    Dataset class specialization to be used by a Pytorch dataloader.
    
    Attributes
    ----------
    X : np.ndarray
        input data points.
    y : np.ndarray
        labels corresponding to input data points
    """
    def __init__(self, X, y):
        self.X = copy.deepcopy(X)
        self.y = copy.deepcopy(y)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, int]:
        """Return data point, label couple in position idx.

        Args:
            idx (int): current index.

        Returns:
            Tuple[torch.Tensor, int]: data point, label couple in position idx.
        """
        return torch.from_numpy(self.X[idx]).float(), self.y[idx]

def evaluate_predictions(output: torch.Tensor, target: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    """Compute matching between predictions array and the target array.

    Args:
        output (torch.Tensor): output predictions.
        target (torch.Tensor): ground truth.
    
    Returns:
        Tuple[torch.Tensor, torch.Tensor]: outputs, counts of matching predictions per output.
    """
    pred = torch.reshape(torch.topk(output, 1)[1], (-1,))
    return torch.unique(pred[pred == target], return_counts=True)
    
def train_one_epoch(
    model: nn.Module,
    optimizer: optim.Optimizer,
    loss_fn: nn.Module,
    training_loader: torch.utils.data.DataLoader,
    batch_size: int = 250
) -> Tuple[float, float]:
    """Epoch training step.

    Args:
        model (nn.Module): input model to be trained.
        optimizer (optim.Optimizer): loss function optimizer.
        loss_fn (nn.Module): loss function.
        training_loader (torch.utils.data.DataLoader): training set data loader.
        batch_size (int): size of every training.
    
    Returns:
        Tuple[int, int]: average training loss, average training accuracy.
    """
    running_loss = 0.
    last_loss = 0.
    running_accuracy = 0.
    last_accuracy = 0.

    for i, data in enumerate(training_loader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        running_accuracy += evaluate_predictions(outputs, labels)[1].sum().item() / batch_size
        
        if i % len(training_loader) == len(training_loader) - 1:
            last_loss = running_loss / len(training_loader) # loss per batch
            last_accuracy = running_accuracy / len(training_loader)
            print('  batch {} loss: {}'.format(i + 1, last_loss))

    return last_loss, last_accuracy
    
def train(
    model: nn.Module,
    epochs: int,
    optimizer: optim.Optimizer,
    loss_fn: nn.Module,
    training_loader: torch.utils.data.DataLoader,
    batch_size: int = 250,
    patience: int = 20
) -> Dict[str, list[float]]:
    """Epochs training scheduler.

    Args:
        model (nn.Module): input model to be trained.
        optimizer (optim.Optimizer): loss function optimizer.
        lr_scheduler (optim.lr_scheduler.LRScheduler): learning rate scheduler.
        loss_fn (nn.Module): loss function.
        training_loader (torch.utils.data.DataLoader): training set data loader.
        batch_size (int): size of every training batch.

    Returns:
        Dict[str, list[float]]: average training / validation loss history, average training / validation accuracy history.
    """
    if os.path.exists(OUTPUT_PATH / 'models_pytorch'):
        shutil.rmtree(OUTPUT_PATH / 'models_pytorch')
    os.mkdir(OUTPUT_PATH / 'models_pytorch')
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    early_stop = 0
    best_vloss = 1000000.
    history_vloss = []
    history_vaccuracy = []
    history_loss = []
    history_accuracy = []
    
    for epoch in range(epochs):
        print('EPOCH {}:'.format(epoch + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss, avg_accuracy = train_one_epoch(model, optimizer, criterion, train_dataloader, batch_size)
        
        history_loss.append(avg_loss)
        history_accuracy.append(avg_accuracy)
        running_vloss = 0.0
        running_vaccuracy = 0.0
        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(valid_dataloader):
                vinputs, vlabels = vdata
                voutputs = model(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss
                running_vaccuracy += evaluate_predictions(voutputs, vlabels)[1].sum().item() / len(vlabels)

            avg_vloss = running_vloss / len(valid_dataloader)
            avg_vaccuracy = running_vaccuracy / len(valid_dataloader)
            history_vloss.append(avg_vloss)
            history_vaccuracy.append(avg_vaccuracy)

        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
        print('ACCURACY train {} valid {}'.format(avg_accuracy, avg_vaccuracy))

        early_stop += 1
        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = OUTPUT_PATH / 'models_pytorch' / f'dense_{timestamp}_{epoch}'
            torch.save(model.state_dict(), model_path)
            early_stop = 0

        if early_stop == patience:
            break

    # Note: add Softmax layer to have class probabilities in output during inference
    model.add_module(f'{len(model)}', nn.Softmax(dim=1))

    model.load_state_dict(torch.load(model_path))
    torch.onnx.export(
        model, torch.from_numpy(np.array([X_test[0]])).float(), f=OUTPUT_PATH / MODEL_NAME,
        export_params = True
    )
    shutil.rmtree(OUTPUT_PATH / 'models_pytorch')

    return {'loss': history_loss, 'val_loss': history_vloss, 'accuracy': history_accuracy, 'val_accuracy': history_vaccuracy}

<p>Define data visualization utilities</p>

In [ ]:
def plot_model_history(model_name: str, history: Dict[str, np.ndarray]) -> None:
    """Plot model training history.

    Args:
        model_name (str): Keras model instance.
        history (Dict[str, np.ndarray]): Training history instance.
    """
    _, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
    for metric, ax in zip(["accuracy", "loss"], axes):
        ax.set_title(f"{model_name} {metric}")
        ax.plot(history[f"{metric}"])
        ax.plot(history[f"val_{metric}"])
        ax.set_ylabel(f"{metric}")
        ax.set_xlabel("epoch")
        ax.legend(["train", "validation"], loc="upper left")

def plot_confusion_matrix(
    y_test: np.ndarray,
    y_pred: np.ndarray,
    labels_dict: Dict[int, str],
    title: Optional[str] = None,
) -> None:
    """Plot confusion matrix with given class names.

    Args:
        y_test (np.ndarray): True labels.
        y_pred (np.ndarray): Predicted labels.
        labels_dict (Dict[int, str]): Class value to semantic label mapping.
        title (str | None): Plot figure title. Defaults to None.
    """
    _, ax = plt.subplots(1, 1, figsize=(6, 5))
    ax = plt.subplot(1, 1, 1)
    if title is None:
        title = "Confusion Matrix"
    ax.set_title(title)
    cm = confusion_matrix(y_test, y_pred, normalize="true")
    labels = list(labels_dict.values())[:len(cm)]
    df_cm = pd.DataFrame(cm, labels, labels)
    sns.heatmap(df_cm, ax=ax, annot=True, annot_kws={"size": 12}, fmt=".2f")
    plt.show()

def plot_features_distribution(
    X: np.ndarray,
    y: np.ndarray,
    features_dict: Dict[int, str],
    labels_dict: Dict[int, str],
    n_rows: int,
    n_cols: int
) -> None:
    """Plot the features distribution over target classes.

    Args:
        X (np.ndarray): data points.
        y (np.ndarray): labels corresponding to data points.
        features_dict (Dict[int, str]): mapping between features index and features semantic.
        labels_dict (Dict[int, str]): mapping between labels index and labels semantic.
        n_rows (int): number of rows of the subplots.
        n_cols (int): number of columns of the subplots.
    """
    fig, ax = plt.subplots(n_rows, n_cols, sharex=True, figsize=(16, 16))
    fig.suptitle('Normalized features distribution over labels')
    xlabs = [(key, val) for key, val in labels_dict.items()]
    xlabs.sort(key=lambda el: el[0])
    xlabs = [el[1] for el in xlabs]
    row_idx, col_idx = 0, 0

    for feat_idx, feat_name in features_dict.items():
        dist = [[] for i in labels_dict.keys()]
        for data_point, lab in zip(X, y):
            dist[lab].append(data_point[feat_idx])
        ax[row_idx][col_idx].set_title(feat_name)
        ax[row_idx][col_idx].set_xlabel('Label')
        ax[row_idx][col_idx].boxplot(dist, tick_labels=xlabs)
        if col_idx == n_cols - 1:
            row_idx += 1
        col_idx = (col_idx + 1) % n_cols
    plt.show()

<p>Extract, preprocess, and inspect the dataset</p>

In [ ]:
# Load data from CSV logs and segment signals into 2s windows without overlap
X, y, labels_dict, features_dict = load_dataset(DATA_PATH, win_len=52, win_stride=52)

# Split dataset into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3)

# Split dataset into training set and validation set
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.1)

print(f"Training set:   {len(y_train):>4} samples")
print(f"Validation set: {len(y_valid):>4} samples")
print(f"Testing set:    {len(y_test):>4} samples")

# Note: features normalization guarantees high accuracy results both before and after post-training quantization and a smoother training curve
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

train_dataloader = DataLoader(FeaturesDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(FeaturesDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)
valid_dataloader = DataLoader(FeaturesDataset(X_valid, y_valid), batch_size=BATCH_SIZE, shuffle=False)
plot_features_distribution(np.concatenate([X_train, X_valid, X_test]), np.concatenate([y_train, y_valid, y_test]), features_dict, labels_dict, 3, 3)

<p>Define the model and launch the training</p>

In [ ]:
# simple Dense model to classify input data points
model = nn.Sequential(
    nn.Linear(X.shape[1], DENSE_UNITS),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(DENSE_UNITS, DENSE_UNITS // 2),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(DENSE_UNITS // 2, len(set(y))),
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

history = train(
    model,
    EPOCHS,
    optimizer,
    criterion,
    train_dataloader,
    BATCH_SIZE,
    PATIENCE
)

<p>Plot training statistics and inspect input features distribution</p>

In [ ]:
plot_model_history('har_dense', history)

model.eval()
running_taccuracy = 0.
running_tloss = 0.
pred = []
with torch.no_grad():
    for i, tdata in enumerate(test_dataloader):
        tinputs, tlabels = tdata
        toutputs = model(tinputs)
        tloss = criterion(toutputs, tlabels)
        running_tloss += tloss
        running_taccuracy += evaluate_predictions(toutputs, tlabels)[1].sum().item() / len(tlabels)
        pred += list(torch.reshape(torch.topk(toutputs, 1)[1], (-1,)))
avg_taccuracy = running_taccuracy / len(test_dataloader)
print(f'\nPrediction accuracy: {avg_taccuracy:.02%}')
plot_confusion_matrix(
    y_test,
    pred,
    labels_dict,
    f'Confusion matrix: har_dense'
)

<p>Post-training QDQ per-tensor quantization to 8-bit precision</p>

In [ ]:
class CustomDataReader(CalibrationDataReader):
    """
    Custom CalibrationDataReader to feed the post-quantization algorithm with calibration data samples.
    
    Attributes
    ----------
    calibration_data : np.ndarray
        calibration data collection.
    index : int
        cursor to iterate over data points in the calibration data collection.
    """
    def __init__(self, calibration_data):
        self.calibration_data = calibration_data
        self.index = 0

    def get_next(self):
        if self.index < len(self.calibration_data):
            data = np.array([self.calibration_data[self.index]]).astype(np.float32)
            self.index += 1
            return { f'onnx::Gemm_0': data }
        else:
            return None

# Statically quantize the model
data_reader = CustomDataReader(X_train)
quant_pre_process(
    OUTPUT_PATH / MODEL_NAME,
    OUTPUT_PATH / f'q{MODEL_NAME}',
    skip_symbolic_shape=False
)

quantize_static(
    OUTPUT_PATH / MODEL_NAME,
    OUTPUT_PATH / f'q{MODEL_NAME}',
    data_reader,
    quant_format=QuantFormat.QDQ, # Note: this is the recommended format to ensure optimal conversion using stedgeai
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8,
    per_channel=False, # Note: since all the activations are 1-D vectors, use per-tensor quantization to ensure optimal conversion using stedgeai
    nodes_to_exclude=['/7/Softmax'] # Note: avoid quantization of Softmax layer outputs to have floating point probabilities during inference
)

# Run predictions to evaluate accuracy after quantization
ort_int8_sess = InferenceSession(OUTPUT_PATH / f'q{MODEL_NAME}', providers=['CPUExecutionProvider'])
acc = 0.
for sample, label in zip(X_test, y_test):
    ort_inputs = {ort_int8_sess.get_inputs()[0].name: np.array([sample]).astype(np.float32)}
    ort_int8_outs = ort_int8_sess.run(None, ort_inputs)[0]
    if np.argmax(ort_int8_outs) == label:
        acc += 1

print(f'\nQuantized model accuracy: {acc / len(y_test):.02%}')

<p>Save the test set in npz format and print out standard scaler estimated parameters (to be copied-pasted to use them in inference)</p>

In [ ]:
testset = OUTPUT_PATH / "har_testset.npz"
os.makedirs(OUTPUT_PATH, exist_ok=True)
np.savez(testset, m_inputs=X_test, m_outputs=np.eye(len(set(y_test)), dtype='float')[y_test])
print('static const float features_mean[STAI_NETWORK_IN_1_CHANNEL] = {\n\t' + ',\n\t'.join([str(x) + 'f' for x in scaler.mean_]) + '\n};\n')
print('static const float features_inv_std_dev[STAI_NETWORK_IN_1_CHANNEL] = {\n\t' + ',\n\t'.join([str(1 / x) + 'f' for x in scaler.scale_]) + '\n};')